# Java concurrency in practice

## Chapter - 1
## Introduction

### A (very) brief history of concurrency

Writing correct programs is hard; writing correct concurrent programs is harder. There are simply more things that can go wrong in a concurrent program than in a sequential one. So, why do we bother with concurrency? Threads are an inescapable feature of the Java language, and they can simplify the develop- ment of complex systems by turning complicated asynchronous code into simpler straight-line code. In addition, threads are the easiest way to tap the computing power of multiprocessor systems. And, as processor counts increase, exploiting concurrency effectively will only become more important.

Several motivating factors led to the development of operating systems that allowed multiple programs to execute simultaneously:

* __Resource utilization.__ Programs sometimes have to wait for external operations such as input or output, and while waiting can do no useful work. It is more efficient to use that wait time to let another program run.
* __Fairness.__ Multiple users and programs may have equal claims on the machine’s resources. It is preferable to let them share the computer via finer-grained time slicing than to let one program run to completion and then start an- other.
* __Convenience.__ It is often easier or more desirable to write several programs that each perform a single task and have them coordinate with each other as necessary than to write a single program that performs all the tasks.

### Benefits of threads


When used properly, threads can reduce development and maintenance costs and improve the performance of complex applications. Threads make it easier to model how humans work and interact, by turning asynchronous workflows into mostly sequential ones. They can also turn otherwise convoluted code into straight-line code that is easier to write, read, and maintain.

#### Exploiting multiple processors: 
Multiprocessor systems used to be expensive and rare, found only in large data centers and scientific computing facilities. Today they are cheap and plentiful; even low-end server and midrange desktop systems often have multiple proc- essors. This trend will only accelerate; as it gets harder to scale up clock rates, processor manufacturers will instead put more processor cores on a single chip. All the major chip manufacturers have begun this transition, and we are already seeing machines with dramatically higher processor counts.

Since the basic unit of scheduling is the thread, a program with only one thread can run on at most one processor at a time. On a two-processor sys- tem, a single-threaded program is giving up access to half the available CPU resources; on a 100-processor system, it is giving up access to 99%. 

#### Simplicity of modeling
A complicated, asynchronous workflow can be decomposed into a number of simpler, synchronous workflows each running in a separate thread, interacting only with each other at specific synchronization points.

Servlet writers do not need to worry about how many other requests are being processed at the same time or whether the socket input and output streams block; when a servlet’s service method is called in response to a web request, it can process the request synchronously as if it were a single-threaded program.

#### Simplified handling of asynchronous events
A server application that accepts socket connections from multiple remote clients may be easier to develop when each connection is allocated its own thread and allowed to use synchronous I/O.

If an application goes to read from a socket when no data is available, read blocks until some data is available. In a single-threaded application, this means that not only does processing the corresponding request stall, but processing of all requests stalls while the single thread is blocked. To avoid this problem, single- threaded server applications are forced to use nonblocking I/O, which is far more complicated and error-prone than synchronous I/O. However, if each request has its own thread, then blocking does not affect the processing of other requests.

#### More responsive user interfaces 
Modern GUI frameworks, such as the AWT and Swing toolkits, replace the main event loop with an event dispatch thread (EDT). When a user interface event such as a button press occurs, application-defined event handlers are called in the event thread. Most GUI frameworks are single-threaded subsystems, so the main event loop is effectively still present, but it runs in its own thread under the control of the GUI toolkit rather than the application.

### Risks of threads

#### Safety hazards
Thread safety can be unexpectedly subtle because, in the absence of sufficient synchronization, the ordering of operations in multiple threads is unpredictable and sometimes surprising. UnsafeSequence in Listing 1.1, which is supposed to generate a sequence of unique integer values, offers a simple illustration of how the interleaving of actions in multiple threads can lead to undesirable results. It behaves correctly in a single-threaded environment, but in a multithreaded environment does not.

In [1]:
// Without sleep also the program is worng in multithreaded enviroment
public class UnsafeSequence {
    private int value;

    /**
     * Returns a unique value.
     */
    public int getNext() {
        try{
            Thread.currentThread().sleep(10);
        } catch(Exception e){
        }
        return value++;
    }
}

In [2]:
UnsafeSequence unsafeSequence = new UnsafeSequence();
Runnable runnable = () -> {
	System.out.print(unsafeSequence.getNext() + ", ");
};

In [3]:
ExecutorService executor = Executors.newFixedThreadPool(1001);
for (int i = 0; i < 1001; i++) {
	executor.submit(runnable);
}
executor.shutdown();

3, 37, 36, 35, 124, 34, 33, 32, 31, 159, 170, 30, 29, 177, 28, 187, 27, 26, 25, 24, 209, 23, 21, 22, 221, 228, 235, 20, 19, 18, 250, 17, 15, 16, 13, 270, 14, 12, 11, 10, 4, 2, 0, 1, 9, 8, 7, 6, 5, 320, 319, 318, 317, 316, 315, 313, 314, 312, 311, 310, 329, 309, 308, 307, 306, 305, 304, 303, 302, 301, 300, 299, 298, 297, 296, 295, 294, 335, 293, 292, 291, 290, 289, 288, 287, 286, 285, 284, 283, 282, 281, 280, 279, 278, 277, 276, 275, 274, 273, 272, 271, 269, 267, 379, 268, 266, 265, 264, 263, 262, 261, 260, 259, 258, 413, 257, 256, 255, 254, 253, 422, 252, 251, 249, 247, 248, 442, 246, 244, 245, 243, 242, 241, 463, 240, 239, 238, 237, 236, 234, 233, 232, 231, 230, 229, 227, 226, 225, 224, 223, 222, 220, 219, 218, 217, 216, 215, 214, 213, 212, 496, 211, 210, 208, 207, 206, 205, 203, 204, 202, 201, 200, 199, 198, 196, 197, 195, 194, 193, 192, 191, 503, 190, 189, 188, 186, 185, 184, 183, 182, 181, 180, 179, 178, 176, 531, 535, 175, 174, 173, 172, 171, 169, 168, 167, 166, 165, 164, 163, 162

In [4]:
public class Sequence {
    private int nextValue;

    public synchronized int getNext() {
        return nextValue++;
    }
}

In [5]:
Sequence safeSequence = new Sequence();
Runnable runnable_1 = () -> {
	System.out.print(safeSequence.getNext() + ", ");
};

In [6]:
ExecutorService executor_1 = Executors.newFixedThreadPool(1001);
for (int i = 0; i < 1001; i++) {
	executor_1.submit(runnable_1);
}
executor.shutdown();

0, 6, 5, 4, 3, 1, 2, 26, 31, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 73, 12, 11, 10, 9, 8, 7, 91, 90, 89, 88, 87, 86, 107, 85, 115, 84, 83, 82, 81, 80, 79, 143, 78, 77, 76, 75, 74, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 201, 57, 56, 55, 54, 53, 52, 51, 50, 223, 49, 48, 47, 46, 237, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 263, 35, 34, 33, 32, 30, 29, 28, 286, 27, 291, 290, 289, 288, 287, 285, 284, 283, 282, 281, 280, 279, 278, 277, 276, 275, 274, 273, 272, 271, 270, 269, 268, 267, 266, 265, 264, 262, 261, 260, 259, 258, 257, 256, 255, 254, 253, 252, 251, 250, 249, 248, 415, 247, 246, 245, 244, 243, 242, 241, 240, 239, 238, 236, 235, 234, 233, 232, 231, 230, 229, 228, 227, 226, 225, 224, 222, 475, 479, 221, 220, 219, 218, 217, 216, 503, 215, 214, 213, 212, 211, 210, 519, 209, 208, 207, 206, 205, 204, 203, 543, 202, 200, 199, 198, 197, 196, 195, 568, 194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182, 181, 180, 179, 178, 177, 176, 175, 174, 17

#### TODO- Give example with treeset and callable

### Liveness hazards

While safety means “nothing bad ever happens”, liveness concerns the com- plementary goal that “something good eventually happens”. A liveness failure occurs when an activity gets into a state such that it is permanently unable to make forward progress. One form of liveness failure that can occur in sequential programs is an inadvertent infinite loop, where the code that follows the loop never gets executed. The use of threads introduces additional liveness risks. For example, if thread A is waiting for a resource that thread B holds exclusively, and B never releases it, A will wait forever.

Like most concurrency bugs, bugs that cause liveness failures can be elusive because they depend on the rel- ative timing of events in different threads, and therefore do not always manifest themselves in development or testing.

### Performance hazards

Related to liveness is performance. While liveness means that something good even- tually happens, eventually may not be good enough—we often want good things to happen quickly. Performance issues subsume a broad range of problems, in- cluding poor service time, responsiveness, throughput, resource consumption, or scalability. Just as with safety and liveness, multithreaded programs are subject to all the performance hazards of single-threaded programs, and to others as well that are introduced by the use of threads.

## Threads are everywhere
Even if your program never explicitly creates a thread, frameworks may create threads on your behalf, and code called from these threads must be thread-safe. This can place a significant design and implementation burden on developers, since developing thread-safe classes requires more care and analysis than devel- oping non-thread-safe classes.

Every Java application uses threads. When the JVM starts, it creates threads for JVM housekeeping tasks (garbage collection, finalization) and a main thread for running the main method. The AWT (Abstract Window Toolkit) and Swing user interface frameworks create threads for managing user interface events. Tim- er creates threads for executing deferred tasks. Component frameworks, such as servlets and RMI create pools of threads and invoke component methods in these threads.